In [ ]:
import cv2 as cv
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
# Load the video file
cap = cv.VideoCapture("/Users/ashleyson/Desktop/coil/coil1.avi")

# Load the background image
bg = cv.imread("/Users/ashleyson/Desktop/bg.tiff")
bg = cv.cvtColor(bg, cv.COLOR_BGR2GRAY)

In [3]:
while True:
    # Read a frame from the video file
    ret, img1 = cap.read()

    # If the frame was not successfully read, break out of the loop
    if not ret:
        break

    # Convert the frame to grayscale
    img1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)

    # Compute the absolute difference between the background and the frame
    delta = cv.absdiff(img1, bg)

    # Apply morphological opening to the delta image
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (10, 10))
    opened = cv.morphologyEx(delta, cv.MORPH_OPEN, kernel)

    # Apply Gaussian blur to the opened image
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (23, 23))
    blur = cv.GaussianBlur(opened, (23, 23), 0)

    # Apply morphological closing to the blurred image
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (60, 60))
    closed = cv.morphologyEx(blur, cv.MORPH_CLOSE, kernel)

    # Threshold the closed image using Otsu's method
    threshold = cv.threshold(closed, 135, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]

    # Apply the threshold as a mask to the original frame
    x = img1.shape[0]
    y = img1.shape[1]
    for i in range(x):
        for j in range(y):
            if threshold[i][j] == 255:
                img1[i][j] = 0

    # Save the processed frame as a separate image
    cv.imwrite("/Users/ashleyson/Desktop/hi/coil_frame%d.jpg" % cap.get(cv.CAP_PROP_POS_FRAMES), img1)

# Release the video capture object and close all windows
cap.release()
cv.destroyAllWindows()
